## Logging on

Use Selenium to visit https://webapps1.chicago.gov/buildingrecords/ and accept the agreement.

> Think about when you use `.find_element...` and when you use `.find_elementSSS...`

In [1]:
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager

In [2]:
import pandas as pd

import time

from bs4 import BeautifulSoup
import requests

In [3]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\nao22\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
C:\Users\nao22\AppData\Local\Temp/ipykernel_35492/1781563638.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
driver.get("https://webapps1.chicago.gov/buildingrecords/")

In [5]:
driver.find_element(By.XPATH, '/html/body/div/div[4]/form/div[1]/div[1]/input').click()

# //*[@id="agreement"]/div[1]/div[1]
# //*[@id="agreement"]/div[1]/div[2]

In [6]:
driver.find_element(By.XPATH, '/html/body/div/div[4]/form/div[4]/div/button').click()

# /html/body/div/div[4]/form/div[4]/div/button

## Searching

Search for **400 E 41ST ST**.

In [7]:
driver.find_element(By.ID, "fullAddress").send_keys("400 E 41ST ST")


In [8]:
driver.find_element(By.XPATH, '//*[@id="submit"]').click()



## Saving tables with pandas

Use pandas to save a CSV of all **permits** to `Permits - 400 E 41ST ST.csv`. Note that there are **different sections of the page**, not just one long permits table.

> - *Tip: When using `.read_html`, try using `flavor='lxml'` and comparing the results to `flavor='html5lib'`. Which works better?*
> - *Tip: You might need to install `html5lib` using `pip`. If so, you'll need to restart the notebook using **Kernel > Restart** before it will work.*

In [9]:
! pip install lxml html5lib

You should consider upgrading via the 'C:\Users\nao22\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [10]:
table = driver.find_element(By.ID, "resultstable_permits")
table
df = pd.read_html(table.get_attribute('outerHTML'),flavor='lxml')[0]
df.info

<bound method DataFrame.info of      PERMIT #  DATE ISSUED                                DESCRIPTION OF WORK
0   100845718          NaN  ERECT TWO SCAFFOLDS FROM 10/14/2019 TO 10/14/2...
1   100778302          NaN  PERMIT EXPIRES ON 10/17/2018 Erection Starts: ...
2   100721255          NaN  PERMIT EXPIRES ON 10/24/2017 ERECTION STARTS: ...
3   100693399          NaN  INSTALLATION OF LOW VOLTAGE BURGLAR ALARM INTE...
4   100665436          NaN  PERMIT EXPIRES ON 10/24/2016 ERECTION STARTS: ...
5   100610771          NaN  PERMIT EXPIRES ON 10/28/2015 ERECTION STARTS: ...
6   100581991          NaN  TRACE AND REPAIR BROKEN UNDERGROUND FEED TO EX...
7   100479194          NaN     INTERNALLY LIT SIGN CABINET ON SOUTH ELEVATION
8   100385721          NaN  RPACE CONCRETE SLAB WITH NEW AT GROUNGD FLOOR ...
9   100267298          NaN  INTERIOR ALTERATIONS TO MEDICAL OFFICE SUITE 1...
10  100218969          NaN  Revision to Permit. Removing walls at elevator...
11  100195892          NaN  INTE

In [11]:

df2 = pd.read_html(table.get_attribute('outerHTML'), flavor='html5lib')[0]
df2.info
#permit dates are now in df2

<bound method DataFrame.info of      PERMIT # DATE ISSUED                                DESCRIPTION OF WORK
0   100845718  10/11/2019  ERECT TWO SCAFFOLDS FROM 10/14/2019 TO 10/14/2...
1   100778302  08/17/2018  PERMIT EXPIRES ON 10/17/2018 Erection Starts: ...
2   100721255  08/24/2017  PERMIT EXPIRES ON 10/24/2017 ERECTION STARTS: ...
3   100693399  03/03/2017  INSTALLATION OF LOW VOLTAGE BURGLAR ALARM INTE...
4   100665436  08/24/2016  PERMIT EXPIRES ON 10/24/2016 ERECTION STARTS: ...
5   100610771  08/28/2015  PERMIT EXPIRES ON 10/28/2015 ERECTION STARTS: ...
6   100581991  02/18/2015  TRACE AND REPAIR BROKEN UNDERGROUND FEED TO EX...
7   100479194  04/16/2013     INTERNALLY LIT SIGN CABINET ON SOUTH ELEVATION
8   100385721  03/25/2011  RPACE CONCRETE SLAB WITH NEW AT GROUNGD FLOOR ...
9   100267298  12/04/2008  INTERIOR ALTERATIONS TO MEDICAL OFFICE SUITE 1...
10  100218969  01/08/2008  Revision to Permit. Removing walls at elevator...
11  100195892  11/01/2007  INTERIOR ALTERATI

In [12]:
df2.to_csv('Permits - 400 E 41ST ST.csv')

## Saving tables the long way

Save a CSV of all DOB inspections to `Inspections - 400 E 41ST ST.csv`.

This is more complicated than the last one becuse **we also need to save the URL to the inspection** (see how the inspection number is a link?). As a result, you won't be able to use pandas! Instead, you'll need to use a loop and create a list of dictionaries.

You can use Selenium or you can feed the source to BeautifulSoup. You should have approximately 160 rows.

You'll probably need to find the table first, then the rows inside, then the cells inside of each row. You'll probably use lots of list indexing. I might recommend XPath for finding the table.

*Tip: If you get a "list index out of range" error, it's probably due to an issue involving `thead` vs `tbody` elements. What are they? What are they for? What's in them? There are a few ways to troubleshoot it.*

In [86]:
# /html/body/div/div[4]/div[10]/table
doc = BeautifulSoup(driver.page_source)
all_rows = doc.find_all('tbody')[2]('tr')

all_rows

this_table = []

for tr in all_rows:
    dict = {}
    try:
        INSP = tr.select_one('td')
        insp_date = INSP.find_next_sibling('td')
        status = insp_date.find_next_sibling('td')
        dict['INSP'] = INSP.text.replace('\n','')
        dict['url'] = "https://webapps1.chicago.gov" + tr.select_one('a')['href']
        dict['insp_date'] = INSP.find_next_sibling('td')('span')[0].text
        dict['status'] = insp_date.find_next_sibling('td').text
        dict['type_description'] = status.find_next_sibling('td').text
        this_table.append(dict)
    except:
        dict['This row is NaN'] = ("DEPARTMENT OF BUILDINGS INSPECTIONS")
        this_table.append(dict)
        
this_table

[{'INSP': '13587202',
  'url': 'https://webapps1.chicago.gov/buildingrecords/inspectiondetails?addr=364923&insp=13587202',
  'insp_date': '20211025',
  'status': 'FAILED',
  'type_description': 'ANNUAL INSPECTION'},
 {'INSP': '12941650',
  'url': 'https://webapps1.chicago.gov/buildingrecords/inspectiondetails?addr=364923&insp=12941650',
  'insp_date': '20210823',
  'status': 'FAILED',
  'type_description': 'CONSERVATION ANNUAL'},
 {'INSP': '13528258',
  'url': 'https://webapps1.chicago.gov/buildingrecords/inspectiondetails?addr=364923&insp=13528258',
  'insp_date': '20210809',
  'status': 'FAILED',
  'type_description': 'ANNUAL INSPECTION'},
 {'INSP': '13346279',
  'url': 'https://webapps1.chicago.gov/buildingrecords/inspectiondetails?addr=364923&insp=13346279',
  'insp_date': '20210303',
  'status': 'FAILED',
  'type_description': 'ANNUAL INSPECTION'},
 {'INSP': '13175960',
  'url': 'https://webapps1.chicago.gov/buildingrecords/inspectiondetails?addr=364923&insp=13175960',
  'insp_dat

In [105]:
df3 = pd.DataFrame(this_table)
# insp_date will disappear! 

df3['insp_date'] =pd.to_datetime(df3.insp_date, format="%Y/%m/%d")
df3

,INSP,url,insp_date,status,type_description
0,13587202,https://webapps1.chicago.gov/buildingrecords/i...,2021-10-25,FAILED,ANNUAL INSPECTION
1,12941650,https://webapps1.chicago.gov/buildingrecords/i...,2021-08-23,FAILED,CONSERVATION ANNUAL
2,13528258,https://webapps1.chicago.gov/buildingrecords/i...,2021-08-09,FAILED,ANNUAL INSPECTION
3,13346279,https://webapps1.chicago.gov/buildingrecords/i...,2021-03-03,FAILED,ANNUAL INSPECTION
4,13175960,https://webapps1.chicago.gov/buildingrecords/i...,2020-11-30,FAILED,ANNUAL INSPECTION
...,...,...,...,...,...
155,25836,https://webapps1.chicago.gov/buildingrecords/i...,2001-07-09,PASSED,SIGN ANNUAL INSPECTION
156,130126,https://webapps1.chicago.gov/buildingrecords/i...,1997-05-09,CLOSED,FIRE PREVENTION PUMPS LEGACY
157,9475223,https://webapps1.chicago.gov/buildingrecords/i...,1997-01-28,CLOSED,ELEVATOR LEGACY INSPECTION
158,9457580,https://webapps1.chicago.gov/buildingrecords/i...,1997-01-21,CLOSED,ELEVATOR LEGACY INSPECTION


In [107]:
df3.to_csv('Inspections - 400 E 41ST ST.csv')

In [108]:
pd.read_csv('Inspections - 400 E 41ST ST.csv')

,Unnamed: 0,INSP,url,insp_date,status,type_description
0,0,13587202,https://webapps1.chicago.gov/buildingrecords/i...,2021-10-25,FAILED,ANNUAL INSPECTION
1,1,12941650,https://webapps1.chicago.gov/buildingrecords/i...,2021-08-23,FAILED,CONSERVATION ANNUAL
2,2,13528258,https://webapps1.chicago.gov/buildingrecords/i...,2021-08-09,FAILED,ANNUAL INSPECTION
3,3,13346279,https://webapps1.chicago.gov/buildingrecords/i...,2021-03-03,FAILED,ANNUAL INSPECTION
4,4,13175960,https://webapps1.chicago.gov/buildingrecords/i...,2020-11-30,FAILED,ANNUAL INSPECTION
...,...,...,...,...,...,...
155,155,25836,https://webapps1.chicago.gov/buildingrecords/i...,2001-07-09,PASSED,SIGN ANNUAL INSPECTION
156,156,130126,https://webapps1.chicago.gov/buildingrecords/i...,1997-05-09,CLOSED,FIRE PREVENTION PUMPS LEGACY
157,157,9475223,https://webapps1.chicago.gov/buildingrecords/i...,1997-01-28,CLOSED,ELEVATOR LEGACY INSPECTION
158,158,9457580,https://webapps1.chicago.gov/buildingrecords/i...,1997-01-21,CLOSED,ELEVATOR LEGACY INSPECTION


### Bonus preview of Wednesday's content

**You prrrrrobably shouldn't do this one unless you want a real challenge.**

If you click the inspection number, it'll open up a new window that shows you details of the violations from that visit. Count the number of violations for each visit and save it in a new column called **num_violations**.

Save this file as `Inspections - 400 E 41ST ST - with counts.csv`.

Since clicking the link opens in a new window, we have to say "Hey Selenium, pay attention to that new window!" We do that with `driver.switch_to.window(driver.window_handles[-1])` (each window gets a `window_handle`, and we're just asking the driver to switch to the last one.). A rough sketch of what your code will look like is here:

```python
# Click the link that opens the new window

# Switch to the new window/tab
driver.switch_to.window(driver.window_handles[-1])

# Do your scraping in here

# Close the new window/tab
driver.close()

# Switch back to the original window/tab
driver.switch_to.window(driver.window_handles[0])
```

You'll want to play around with them individually before you try it with the whole set - the ones that pass are very different pages than the ones with violations! There are a few ways to get the number of violations, some easier than others.